In [1]:
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
from datasets import Dataset
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('/home/jupyter/datasphere/project/mipt_hackathon.csv')

In [3]:
df.head()

,WikiData,wiki_links,История,Архитектура,llm_text
0,Q37996725,https://ru.wikipedia.org/wiki/%D0%94%D0%B8%D0%...,В сентябре 1929 годаЦК ВКП(б)было принято пост...,"Здание спортклуба, напоминающее корабль, имеет...",### История и Архитектура Здания Спортклуба\n\...
1,Q55209768,https://ru.wikipedia.org/wiki/%D0%94%D0%BE%D0%...,Комплекс зданийДОСААФбыл расположен на месте т...,"Здания клуба, спортивного техникума и жилого д...","### История\n\nКлуб ""Малышева"" - это уникально..."
2,Q55154121,https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%...,В 1894 году выходцы из купеческих семей Андрей...,Комплекс зданий электростанции расположен в це...,Конечно! Вот краткий обзор текста:\n\n**Истори...
3,Q55232375,https://ru.wikipedia.org/wiki/%D0%94%D0%BE%D0%...,Автором проекта являлся городской архитектор Е...,Одноэтажное кирпичное здание с цокольным этажо...,### История и Архитектура Здания\n\n**Здание:*...
4,Q4306077,https://ru.wikipedia.org/wiki/%D0%9B%D0%B8%D1%...,"В 1910—1912 годах дом, в котором теперь распол...",NaN,"Конечно, я помогу вам создать краткое описание..."


In [4]:
import datasets

ds = Dataset.from_pandas(df)

In [5]:
from langchain.docstore.document import Document as LangchainDocument

RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["llm_text"], metadata={"source": doc["WikiData"]}) for doc in tqdm(ds)
]

  0%|          | 0/297 [00:00<?, ?it/s]

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]

In [7]:
EMBEDDING_MODEL_NAME = 'intfloat/multilingual-e5-large'

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer

def split_documents(
    chunk_size: int,
    knowledge_base: List[LangchainDocument],
    tokenizer_name: Optional[str] = EMBEDDING_MODEL_NAME,
) -> List[LangchainDocument]:
    """
    Разобъём документы на блоки максимального размера `chunk_size` и вернём список документов.
    """
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=MARKDOWN_SEPARATORS,
    )

    docs_processed = []
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # Удалим дубли
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique


docs_processed = split_documents(
    1024,  # Выбираем размер чанка, соответствующий модели
    RAW_KNOWLEDGE_BASE,
    tokenizer_name=EMBEDDING_MODEL_NAME,
)

/home/jupyter/.local/lib/python3.8/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [9]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},  # `True` для косинусного сходства
)

KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
    docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
model_name = 'VityaVitalich/Llama3.1-8b-instruct'
# model_name = 'mistralai/Mistral-7B-Instruct-v0.3'
# model_name = 'intfloat/multilingual-e5-small'
# model_name = "BAAI/bge-multilingual-gemma2"

In [12]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quant = True

bnb_config = None
if quant:
    bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16,
  )
    
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map='cuda')
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards: 100%|██████████| 4/4 [05:44<00:00, 86.19s/it] 


In [13]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)

In [14]:
prompt_in_chat_format = [
    {
        "role": "system",
        "content": """Используя информацию, содержащуюся в контексте, 
        предоставьте подробный и содержательный ответ на заданный вопрос. 
        Убедитесь, что ваш ответ охватывает ключевые аспекты темы и включает примеры или пояснения для лучшего понимания. 
        Если контекст не содержит необходимой информации для ответа, 
        используйте свои знания по сути вопроса и дайте обоснованный ответ. 
        Постарайтесь сделать вашу речь живой и увлекательной, 
        чтобы заинтересовать читателя. Отвечайте на русском языке.""",
    },
    {
        "role": "user",
        "content": """Контекст:
{context}
---
Теперь вот вопрос, на который вам нужно ответить.

Вопрос: {question}""",
    },
]
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)
print(RAG_PROMPT_TEMPLATE)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Используя информацию, содержащуюся в контексте, 
        предоставьте подробный и содержательный ответ на заданный вопрос. 
        Убедитесь, что ваш ответ охватывает ключевые аспекты темы и включает примеры или пояснения для лучшего понимания. 
        Если контекст не содержит необходимой информации для ответа, 
        используйте свои знания по сути вопроса и дайте обоснованный ответ. 
        Постарайтесь сделать вашу речь живой и увлекательной, 
        чтобы заинтересовать читателя. Отвечайте на русском языке.<|eot_id|><|start_header_id|>user<|end_header_id|>

Контекст:
{context}
---
Теперь вот вопрос, на который вам нужно ответить.

Вопрос: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [17]:
from transformers import Pipeline

def answer_with_rag(
    question: str,
    llm: Pipeline,
    knowledge_index: FAISS,
    reranker = False,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 5,
) -> Tuple[str, List[LangchainDocument]]:
    # Соберём документы с помощью ретривера
    print("=> Retrieving documents...")
    relevant_docs = knowledge_index.similarity_search(query=question, k=num_retrieved_docs)
    relevant_docs = [doc.page_content for doc in relevant_docs]  # Оставляем только текст

    if reranker:
        print("=> Reranking documents...")
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
        relevant_docs = [doc["content"] for doc in relevant_docs]

    relevant_docs = relevant_docs[:num_docs_final]

    # Финальный промпт
    context = "\nExtracted documents:\n"
    context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)])

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    print("=> Generating answer...")
    answer = llm(final_prompt, pad_token_id=llm.tokenizer.eos_token_id)[0]["generated_text"]

    return answer

In [18]:
question = "Расскажи про Здание спортклуба"

answer = answer_with_rag(question, READER_LLM, KNOWLEDGE_VECTOR_DATABASE, reranker=False)

=> Retrieving documents...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


=> Generating answer...


In [19]:
answer

'Здание спортивного клуба, описанное в документе, представляет собой уникальное сооружение, сочетающее в себе элементы спортивного и культурного назначения. Оно расположено рядом с прудом и является ключевой частью архитектурной среды Екатеринбурга.\n\nАрхитектура спортивного клуба характеризуется традиционным стилем авангардизма и концепта конструктивизма. Здание имеет высокие ярусные балконы, видоискатели в стиле "иллюминаторы", триплетные уровни (главный и две секции) и смотровую площадку. Внутренние помещения включают тренажёрные залы, раздевалки, душевые, массажные кабины, инвентарные помещения и оркестровую комнату на втором этаже. Кроме того, в подвале расположен тир.\n\nФасады спортивного клуба оштукатурены и окрашены в классическом стиле, что соответствует эстетике времени его постройки. Он имеет четыре этажа с выступающим первым и овальным южным фасадом, что делает его легко узнаваемым. \n\nТаким образом, здание спортивного клуба представляет собой уникальное сооружение, объе